## 모델

In [7]:
from sklearn.model_selection import train_test_split

In [1]:
df = pd.read_csv('./MACH_data/cut_off_by_max.csv')
df.head()

,Q1_TP_notell_2u,Q1I,Q1E_notell_2u,Q2_TP_ppl_nd_dangun,Q2I,Q2E_ppl_nd_dangun,Q3_TN_do_moral,Q3I,Q3E_do_moral,Q4_VN_ppl_good,...,religion,orientation,race,voted,married,familysize,score,v_score,t_score,m_score
0,3.0,6.0,21.0,3.0,7.0,19.0,5.0,20.0,15.0,2.0,...,Christian(othrer),Heterosexual,Black,Yes,Currently married,5,75.0,35.0,33.0,7.0
1,5.0,17.0,4.0,5.0,9.0,8.0,1.0,16.0,6.0,3.0,...,Agnostic,Heterosexual,White,No,Never married,2,91.0,37.0,45.0,9.0
2,5.0,16.0,4.0,5.0,12.0,3.0,1.0,2.0,7.0,1.0,...,Atheist,Bisexual,Asian,No,Never married,2,98.0,43.0,45.0,10.0
3,2.0,12.0,9.0,4.0,1.0,10.0,2.0,7.0,10.0,1.0,...,Christian(Protestant),Heterosexual,White,Yes,Previously married,2,78.0,37.0,36.0,5.0
4,5.0,13.0,9.0,5.0,7.0,5.0,2.0,19.0,9.0,1.0,...,Christian(Catholic),Homosexual,White,Yes,Never married,2,85.0,35.0,41.0,9.0


In [8]:
auto_train, auto_test = train_test_split(df, test_size=0.2,
                                          random_state=13)

In [9]:
auto_train.shape, auto_test.shape

((53517, 108), (13380, 108))

### auto ml

In [11]:
from pycaret.classification import *

In [12]:
# 'voted' 컬럼이 예측 대상이므로 target 인자에 명시
# 'voted' column is the target variable
clf = setup(data = auto_train, target = 'voted')

Setup Succesfully Completed!


,Description,Value
0,session_id,8968
1,Target Type,Binary
2,Label Encoded,"No: 0, Yes: 1"
3,Original Data,"(53517, 108)"
4,Missing Values,False
5,Numeric Features,71
6,Categorical Features,36
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [13]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,CatBoost Classifier,0.7036,0.7870,0.7346,0.6469,0.6879,0.4078,0.4109,10.9192
1,Light Gradient Boosting Machine,0.6990,0.7834,0.7383,0.6402,0.6857,0.3997,0.4036,1.1791
2,Gradient Boosting Classifier,0.6956,0.7795,0.7412,0.6353,0.6841,0.3938,0.3982,24.0500
3,Ada Boost Classifier,0.6953,0.7763,0.7366,0.6360,0.6825,0.3926,0.3966,6.4958
4,Extreme Gradient Boosting,0.6885,0.7703,0.7005,0.6362,0.6667,0.3757,0.3772,47.5566
5,Linear Discriminant Analysis,0.6913,0.7628,0.6417,0.6566,0.6490,0.3735,0.3736,1.5261
6,Extra Trees Classifier,0.6816,0.7575,0.6179,0.6494,0.6332,0.3523,0.3527,3.0659
7,Random Forest Classifier,0.6421,0.7153,0.5007,0.6212,0.5544,0.2611,0.2654,0.3510
8,Naive Bayes,0.4499,0.6438,0.9813,0.4463,0.6135,0.0050,0.0119,0.1649
9,Logistic Regression,0.5949,0.6200,0.2450,0.5694,0.3253,0.1263,0.1448,0.8875


In [14]:
best_3

 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                random_state=8968, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='deviance', max_depth=3,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=100,
                            n_iter_no_change=None, presort='deprecated',
           

### automl 앙상블

In [15]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7011,0.7866,0.7384,0.6427,0.6872,0.4036,0.4071
1,0.7032,0.7878,0.7423,0.6444,0.6899,0.4079,0.4117
2,0.7074,0.7898,0.7407,0.6502,0.6925,0.4157,0.4189
3,0.7038,0.7855,0.7557,0.6419,0.6941,0.4107,0.4159
4,0.6995,0.7843,0.7377,0.6409,0.6859,0.4007,0.4043
Mean,0.7030,0.7868,0.7429,0.6440,0.6899,0.4077,0.4116
SD,0.0027,0.0019,0.0066,0.0033,0.0031,0.0053,0.0054


In [16]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7015,0.786,0.738,0.6434,0.6874,0.4043,0.4078


In [17]:
final_model = finalize_model(blended)

In [18]:
predictions = predict_model(final_model, data = auto_test)

In [22]:
predictions.isnull().sum()


Q1_TP_notell_2u        10723
Q1I                    10723
Q1E_notell_2u          10723
Q2_TP_ppl_nd_dangun    10723
Q2I                    10723
                       ...  
v_score                10723
t_score                10723
m_score                10723
Label                  10723
Score                  10723
Length: 110, dtype: int64

In [21]:
predictions

,Q1_TP_notell_2u,Q1I,Q1E_notell_2u,Q2_TP_ppl_nd_dangun,Q2I,Q2E_ppl_nd_dangun,Q3_TN_do_moral,Q3I,Q3E_do_moral,Q4_VN_ppl_good,...,race,voted,married,familysize,score,v_score,t_score,m_score,Label,Score
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0133
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6202
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1890
3,2.0,12.0,9.0,4.0,1.0,10.0,2.0,7.0,10.0,1.0,...,White,Yes,Previously married,2.0,78.0,37.0,36.0,5.0,Yes,0.6192
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66875,2.0,3.0,10.0,2.0,11.0,10.0,4.0,9.0,17.0,3.0,...,White,No,Never married,3.0,51.0,27.0,19.0,5.0,No,NaN
66876,2.0,15.0,17.0,3.0,19.0,9.0,3.0,14.0,8.0,4.0,...,White,No,Currently married,3.0,47.0,19.0,23.0,5.0,Yes,NaN
66883,2.0,20.0,6.0,3.0,14.0,4.0,2.0,10.0,7.0,3.0,...,White,No,Never married,2.0,68.0,31.0,30.0,7.0,Yes,NaN
66885,3.0,20.0,11.0,1.0,2.0,28.0,4.0,3.0,32.0,2.0,...,White,No,Never married,4.0,50.0,26.0,19.0,5.0,Yes,NaN
